**Import required packages**

In [1]:
import pandas as pd
import json 
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import seaborn as sns
import numpy as np
from numpy.random import dirichlet
from ema_workbench import TimeSeriesOutcome, Model, perform_experiments, RealParameter, Scenario, CategoricalParameter, ScalarOutcome, ema_logging, save_results, Policy
from ema_workbench.em_framework.samplers import LHSSampler  
from ema_workbench import SequentialEvaluator
from ema_workbench.analysis import lines, prim
from ema_workbench.analysis.plotting_util import Density
from ema_workbench.connectors.vensim import VensimModel

C:\Users\saski\anaconda3\Lib\site-packages\ema_workbench\connectors\__init__.py:29: ImportWarning: netlogo connector not available
  warnings.warn("netlogo connector not available", ImportWarning)
C:\Users\saski\anaconda3\Lib\site-packages\ema_workbench\connectors\__init__.py:34: ImportWarning: simio connector not available
  warnings.warn("simio connector not available", ImportWarning)


The reference scenario already has the correct parameter values set in the Vensim model. So we proceed by loading the model.

**Load model**

In [5]:
 #turn on logging
ema_logging.log_to_stderr(ema_logging.INFO)

#initialize the Vensim model
#wd = r"C:\Users\saski\OneDrive\Documenten\EPA\AFSTUDEREN\Model\Definitive Model Version"
#vensimModel = VensimModel("FuturePredictionModel", wd=wd, model_file="Model Future Prediction 17.1 FINAL.vpmx")
model_directory = r"C:\Users\saski\OneDrive\Documenten\EPA\AFSTUDEREN\Model\Definitive Model Version"
model_file = "ModelReboundSA.vpmx"
vensim_model = VensimModel("SAmodel", wd=model_directory, model_file=model_file)

#vensim_model.sampler = None

In [7]:
haul_segments = ["ShortHaul", "ShortMediumHaul", "MediumHaul", "LongHaul", "UltraLongHaul"]

n_experiments = 1

scenarios = []

elasticity_values = {
    "ShortHaul": -1.28,
    "ShortMediumHaul": -1.28,
    "MediumHaul": -1.06,
    "LongHaul": -0.83,
    "UltraLongHaul": -0.83
}

for i in range(n_experiments):
    scenario = Scenario()
    
    # Add market share values for each segment
    for segment in haul_segments:
        scenario[f"Market share per haul segment[{segment}]"] = 0.2
        scenario[f"Fare elasticity of demand[{segment}]"] = elasticity_values[segment]
        scenario[f"Pricing strategy factor[{segment}]"] = 0.45
    
    scenarios.append(scenario)
    
for scenario in scenarios:
    print(scenario)

Scenario({'Market share per haul segment[ShortHaul]': 0.2, 'Fare elasticity of demand[ShortHaul]': -1.28, 'Pricing strategy factor[ShortHaul]': 0.45, 'Market share per haul segment[ShortMediumHaul]': 0.2, 'Fare elasticity of demand[ShortMediumHaul]': -1.28, 'Pricing strategy factor[ShortMediumHaul]': 0.45, 'Market share per haul segment[MediumHaul]': 0.2, 'Fare elasticity of demand[MediumHaul]': -1.06, 'Pricing strategy factor[MediumHaul]': 0.45, 'Market share per haul segment[LongHaul]': 0.2, 'Fare elasticity of demand[LongHaul]': -0.83, 'Pricing strategy factor[LongHaul]': 0.45, 'Market share per haul segment[UltraLongHaul]': 0.2, 'Fare elasticity of demand[UltraLongHaul]': -0.83, 'Pricing strategy factor[UltraLongHaul]': 0.45})


In [16]:
uncertain_parameters = []

for segment in haul_segments:
    uncertain_parameters.append(
            RealParameter(f"Market share per haul segment[{segment}]", 0.1999, 0.2001)
    )
        
    #add uncertain parameters for fare elasticity and pricing strategy factor
    uncertain_parameters.append(
        RealParameter(f"Fare elasticity of demand[{segment}]", -1.28, -0.83)
    )
    uncertain_parameters.append(
        RealParameter(f"Pricing strategy factor[{segment}]", 0.4449, 0.4501)
    )
    
vensim_model.uncertainties = uncertain_parameters

**Define KPIs**

In [8]:
vensim_model.outcomes = [TimeSeriesOutcome("Total accumulated emissions", variable_name="Total accumulated emissions"),
                        TimeSeriesOutcome("Total annual passenger demand", variable_name="Total annual passenger demand")]

**Perform one single run**

In [17]:
from ema_workbench import SequentialEvaluator

experiment, outcome_refscenario = perform_experiments(vensim_model, scenarios=n_experiments) 

[MainProcess/INFO] performing 1 scenarios * 1 policies * 1 model(s) = 1 experiments


  0%|                                                    | 0/1 [00:00<?, ?it/s][MainProcess/INFO] performing experiments sequentially


100%|████████████████████████████████████████████| 1/1 [00:00<00:00,  5.77it/s]
[MainProcess/INFO] experiments finished


In [18]:
save_results((experiment, outcome_refscenario), "./results/results_refscenario.tar.gz")

C:\Users\saski\anaconda3\Lib\site-packages\ema_workbench\em_framework\outcomes.py:541: UserWarning: still to be tested!!
  warnings.warn("still to be tested!!")
[MainProcess/INFO] results saved successfully to C:\Users\saski\OneDrive\Documenten\EPA\AFSTUDEREN\results\results_refscenario.tar.gz
